In [1]:
import os
import numpy as np
import pandas as pd
import SimpleITK as sitk
from sklearn.metrics import jaccard_score
# from scipy.spatial.distance import directed_hausdorff
from medpy.metric import binary
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.structures import Meshes
from skimage import measure
from pytorch3d.structures import Meshes

def mask_to_mesh(mask):
    # Use marching cubes to obtain the surface mesh of the mask
    verts, faces, normals, values = measure.marching_cubes_lewiner(mask)

    # Convert verts and faces to PyTorch tensors
    verts_tensor = torch.from_numpy(verts).float()
    faces_tensor = torch.from_numpy(faces)

    # Create a Meshes object
    mesh = Meshes(verts=[verts_tensor], faces=[faces_tensor])

    return mesh

# In your calculate_surf_dsc function:
def calculate_surf_dsc(label_mask, DL_mask):
    # convert masks to Meshes
    label_mesh = mask_to_mesh(label_mask)
    DL_mesh = mask_to_mesh(DL_mask)

    # sample points on the surface of the meshes
    label_points = sample_points_from_meshes(label_mesh, 1000)
    DL_points = sample_points_from_meshes(DL_mesh, 1000)

    # calculate average path length
    dist_matrix = np.linalg.norm(label_points - DL_points[:, None], axis=-1)
    apl = np.mean(np.min(dist_matrix, axis=0))

    return apl

def calculate_metrics(patient_directory):
    metrics = []

    for root, dirs, files in os.walk(patient_directory):
        if 'label_mask.nii' in files and 'DL_mask.nrrd' in files:
            
            # Load ground truth and prediction
            label_mask = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(root, 'label_mask.nii')))
            DL_mask = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(root, 'DL_mask.nrrd')))

            # Compute Dice score
            intersect = np.sum(label_mask * DL_mask)
            dice_score = 2. * intersect / (np.sum(label_mask) + np.sum(DL_mask))
            
            # Compute Jaccard score
            jaccard_score = intersect / (np.sum(label_mask) + np.sum(DL_mask) - intersect)
            
#             # Compute 95th percentile Hausdorff distance
#             h95th_score = binary.hd95(label_mask, DL_mask)
            
            # Compute Surface Dice score
#             surf_dsc_score = calculate_surf_dsc(label_mask, DL_mask)
            
            # Compute the Average Path Length (APL)
#             apl_score = calculate_apl(label_mask, DL_mask)

            # Assume folder structure as "patient_directory/patient_id-exam_id-exam_date/"
            print(root)
            patient_id, exam_id, exam_date = os.path.basename(root).split("-")

            metrics.append({
                "patient_id": patient_id,
                "exam_id": exam_id,
                "exam_date": exam_date,
                "3d_dice": dice_score,
                "3d_jacc": jaccard_score,
#                 "h95th": h95th_score,
#                 "surf_dsc": surf_dsc_score,
#                 "APL": apl_score,
            })

    return pd.DataFrame(metrics)

# use the function
patient_directory = "../Automatic segmentation script/patient_segmentations"
metrics_df = calculate_metrics(patient_directory)

# save the dataframe to a csv file
metrics_df.to_csv('metrics.csv', index=False)

# print the dataframe
print(metrics_df)


/opt/anaconda3/envs/nsclc_seg/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


../Automatic segmentation script/patient_segmentations/YG_HWSCOBELFJHZ-YG_DGNWZ1SLBAU5-06_26_2019
../Automatic segmentation script/patient_segmentations/YG_HWSCOBELFJHZ-YG_5QRI1MX3J83R-12_19_2022
../Automatic segmentation script/patient_segmentations/YG_HWSCOBELFJHZ-YG_7EBLNXLZDNOY-10_20_2021
        patient_id          exam_id   exam_date   3d_dice   3d_jacc
0  YG_HWSCOBELFJHZ  YG_DGNWZ1SLBAU5  06_26_2019  0.666267  0.499550
1  YG_HWSCOBELFJHZ  YG_5QRI1MX3J83R  12_19_2022  0.708617  0.548728
2  YG_HWSCOBELFJHZ  YG_7EBLNXLZDNOY  10_20_2021  0.650614  0.482155
